# 2.1 Escolher um servidor SGBDR a ser utilizado

My SQL

# 2.2 Baixar, instalar localmente (localhost) e configurar o servidor de banco de dados, ou configurar o serviço do servidor de banco de dados em nuvem (DBaaS)

Feito!

# 2.3 Configurar o serviço de segurança do banco de dados (roles, usuários e permissões)

Pendente....

# 2.4 Baixar os dataset para o projeto e prepará-lo para carga no banco de dados. Esta preparação poderá ser feita utilizando SQL em área de staging do servidor SGBDR. Se a opção for esta última, então haverá a atividade de criação do raw schema da área de staging.

Pendente... 

Douglas precisa complementar especificando passo a passo o que foi feito.

# 2.5 Projetar a aplicação

### 2.5.1 Projeto funcional da aplicação

(i) Módulo para a geração de datasets para processos de análise

In [ ]:
import pandas as pd
import sqlite3

# Conectar ao banco de dados
conn = sqlite3.connect('eleicoes.db')

def gerar_dataset(query):
    # Executar a consulta e retornar o resultado como um DataFrame
    df = pd.read_sql_query(query, conn)
    return df

# Exemplo de uso
query = """
SELECT * 
FROM PERFIL_ELEITORADO 
WHERE ANO_ELEICAO = 2024
"""
dataset = gerar_dataset(query)
print(dataset.head())

(ii) Módulo para a construção dinâmica de gráficos analíticos

In [ ]:
import matplotlib.pyplot as plt

def construir_grafico(df, x_col, y_col, grafico_tipo='bar'):
    if grafico_tipo == 'bar':
        df.plot(kind='bar', x=x_col, y=y_col)
    elif grafico_tipo == 'line':
        df.plot(kind='line', x=x_col, y=y_col)
    plt.show()

# Exemplo de uso
construir_grafico(dataset, 'SG_UF', 'QT_ELEITORES_PERFIL', 'bar')

### 2.5.2 Projeto do banco de dados

(i) Modelagem do banco de dados até o nível físico

In [ ]:
-- A tabela já está criada conforme fornecido:
CREATE TABLE IF NOT EXISTS PERFIL_ELEITORADO (
    DT_GERACAO DATE,
    ANO_ELEICAO INT,
    SG_UF VARCHAR(2),
    NM_MUNICIPIO VARCHAR(100),
    NR_ZONA INT,
    CD_GENERO INT,
    CD_ESTADO_CIVIL INT,
    CD_FAIXA_ETARIA INT,
    CD_GRAU_ESCOLARIDADE INT,
    CD_RACA_COR INT,
    CD_IDENTIDADE_GENERO INT,
    CD_QUILOMBOLA INT,
    CD_INTERPRETE_LIBRAS INT,
    QT_ELEITORES_PERFIL INT,
    QT_ELEITORES_BIOMETRIA INT,
    QT_ELEITORES_DEFICIENCIA INT,
    QT_ELEITORES_INC_NM_SOCIAL INT
);

(ii) Especificação das consultas analíticas e do Nível de Visão (Views)

In [ ]:
-- View para total de eleitores por estado
CREATE VIEW total_eleitores_estado AS
SELECT 
    SG_UF, 
    SUM(QT_ELEITORES_PERFIL) AS total_eleitores
FROM 
    PERFIL_ELEITORADO
GROUP BY 
    SG_UF;

-- Consulta para extrair dados da View
SELECT * FROM total_eleitores_estado;

In [ ]:
-- View para total de eleitores por município
CREATE VIEW IF NOT EXISTS total_eleitores_municipio AS
SELECT 
    NM_MUNICIPIO, 
    SUM(QT_ELEITORES_PERFIL) AS total_eleitores
FROM 
    PERFIL_ELEITORADO
GROUP BY 
    NM_MUNICIPIO;

In [ ]:
-- View para total de eleitores por faixa etária
CREATE VIEW IF NOT EXISTS total_eleitores_faixa_etaria AS
SELECT 
    CD_FAIXA_ETARIA, 
    SUM(QT_ELEITORES_PERFIL) AS total_eleitores
FROM 
    PERFIL_ELEITORADO
GROUP BY 
    CD_FAIXA_ETARIA;

In [ ]:
-- View para total de eleitores por grau de escolaridade
CREATE VIEW IF NOT EXISTS total_eleitores_escolaridade AS
SELECT 
    CD_GRAU_ESCOLARIDADE, 
    SUM(QT_ELEITORES_PERFIL) AS total_eleitores
FROM 
    PERFIL_ELEITORADO
GROUP BY 
    CD_GRAU_ESCOLARIDADE;

In [ ]:
-- View para total de eleitores por raça/cor
CREATE VIEW IF NOT EXISTS total_eleitores_raca_cor AS
SELECT 
    CD_RACA_COR, 
    SUM(QT_ELEITORES_PERFIL) AS total_eleitores
FROM 
    PERFIL_ELEITORADO
GROUP BY 
    CD_RACA_COR;

In [ ]:
-- View para total de eleitores com deficiência
CREATE VIEW IF NOT EXISTS total_eleitores_deficiencia AS
SELECT 
    SUM(QT_ELEITORES_DEFICIENCIA) AS total_eleitores_deficiencia
FROM 
    PERFIL_ELEITORADO;

(iii) Projeto dos demais objetos de banco

In [ ]:
-- Índices para melhorar o desempenho das consultas
CREATE INDEX idx_eleitores_uf ON PERFIL_ELEITORADO (SG_UF);
CREATE INDEX idx_eleitores_ano ON PERFIL_ELEITORADO (ANO_ELEICAO);

-- Trigger para atualizar uma tabela de log sempre que um perfil de eleitorado for inserido
CREATE TABLE IF NOT EXISTS log_insercao (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    log_data TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    SG_UF VARCHAR(2),
    NM_MUNICIPIO VARCHAR(100),
    ANO_ELEICAO INT
);

CREATE TRIGGER log_insercao_perfil
AFTER INSERT ON PERFIL_ELEITORADO
BEGIN
    INSERT INTO log_insercao (SG_UF, NM_MUNICIPIO, ANO_ELEICAO)
    VALUES (NEW.SG_UF, NEW.NM_MUNICIPIO, NEW.ANO_ELEICAO);
END;

-- Stored Procedure para inserir dados no perfil de eleitorado
CREATE PROCEDURE insere_perfil_eleitorado (
    IN p_DT_GERACAO DATE,
    IN p_ANO_ELEICAO INT,
    IN p_SG_UF VARCHAR(2),
    IN p_NM_MUNICIPIO VARCHAR(100),
    IN p_NR_ZONA INT,
    IN p_CD_GENERO INT,
    IN p_CD_ESTADO_CIVIL INT,
    IN p_CD_FAIXA_ETARIA INT,
    IN p_CD_GRAU_ESCOLARIDADE INT,
    IN p_CD_RACA_COR INT,
    IN p_CD_IDENTIDADE_GENERO INT,
    IN p_CD_QUILOMBOLA INT,
    IN p_CD_INTERPRETE_LIBRAS INT,
    IN p_QT_ELEITORES_PERFIL INT,
    IN p_QT_ELEITORES_BIOMETRIA INT,
    IN p_QT_ELEITORES_DEFICIENCIA INT,
    IN p_QT_ELEITORES_INC_NM_SOCIAL INT
)
BEGIN
    INSERT INTO PERFIL_ELEITORADO (
        DT_GERACAO, ANO_ELEICAO, SG_UF, NM_MUNICIPIO, NR_ZONA, CD_GENERO, 
        CD_ESTADO_CIVIL, CD_FAIXA_ETARIA, CD_GRAU_ESCOLARIDADE, CD_RACA_COR, 
        CD_IDENTIDADE_GENERO, CD_QUILOMBOLA, CD_INTERPRETE_LIBRAS, 
        QT_ELEITORES_PERFIL, QT_ELEITORES_BIOMETRIA, QT_ELEITORES_DEFICIENCIA, 
        QT_ELEITORES_INC_NM_SOCIAL
    )
    VALUES (
        p_DT_GERACAO, p_ANO_ELEICAO, p_SG_UF, p_NM_MUNICIPIO, p_NR_ZONA, 
        p_CD_GENERO, p_CD_ESTADO_CIVIL, p_CD_FAIXA_ETARIA, p_CD_GRAU_ESCOLARIDADE, 
        p_CD_RACA_COR, p_CD_IDENTIDADE_GENERO, p_CD_QUILOMBOLA, p_CD_INTERPRETE_LIBRAS, 
        p_QT_ELEITORES_PERFIL, p_QT_ELEITORES_BIOMETRIA, p_QT_ELEITORES_DEFICIENCIA, 
        p_QT_ELEITORES_INC_NM_SOCIAL
    );
END;


### 2.5.3 Projetos dos testes e validações

Testes em Python

In [ ]:
def test_gerar_dataset():
    query = "SELECT * FROM PERFIL_ELEITORADO WHERE ANO_ELEICAO = 2024"
    dataset = gerar_dataset(query)
    assert not dataset.empty, "Dataset should not be empty"

def test_construir_grafico():
    df = pd.DataFrame({
        'SG_UF': ['SP', 'RJ', 'MG'],
        'QT_ELEITORES_PERFIL': [1000000, 800000, 600000]
    })
    try:
        construir_grafico(df, 'SG_UF', 'QT_ELEITORES_PERFIL', 'bar')
    except Exception as e:
        assert False, f"Grafico falhou com erro: {e}"

# Executar testes
test_gerar_dataset()
test_construir_grafico()

Testes em SQL

In [ ]:
-- Testar inserção de dados na tabela PERFIL_ELEITORADO
INSERT INTO PERFIL_ELEITORADO (
    DT_GERACAO, ANO_ELEICAO, SG_UF, NM_MUNICIPIO, NR_ZONA, CD_GENERO, 
    CD_ESTADO_CIVIL, CD_FAIXA_ETARIA, CD_GRAU_ESCOLARIDADE, CD_RACA_COR, 
    CD_IDENTIDADE_GENERO, CD_QUILOMBOLA, CD_INTERPRETE_LIBRAS, 
    QT_ELEITORES_PERFIL, QT_ELEITORES_BIOMETRIA, QT_ELEITORES_DEFICIENCIA, 
    QT_ELEITORES_INC_NM_SOCIAL
) VALUES (
    '2024-07-16', 2024, 'SP', 'Sao Paulo', 1, 1, 1, 1, 1, 1, 1, 1, 1, 
    1000, 500, 100, 50
);

SELECT * FROM PERFIL_ELEITORADO WHERE SG_UF = 'SP';

-- Testar View de total de eleitores por estado
SELECT * FROM total_eleitores_estado WHERE SG_UF = 'SP';

-- Verificar se Trigger está funcionando corretamente
INSERT INTO PERFIL_ELEITORADO (
    DT_GERACAO, ANO_ELEICAO, SG_UF, NM_MUNICIPIO, NR_ZONA, CD_GENERO, 
    CD_ESTADO_CIVIL, CD_FAIXA_ETARIA, CD_GRAU_ESCOLARIDADE, CD_RACA_COR, 
    CD_IDENTIDADE_GENERO, CD_QUILOMBOLA, CD_INTERPRETE_LIBRAS, 
    QT_ELEITORES_PERFIL, QT_ELEITORES_BIOMETRIA, QT_ELEITORES_DEFICIENCIA, 
    QT_ELEITORES_INC_NM_SOCIAL
) VALUES (
    '2024-07-16', 2024, 'RJ', 'Rio de Janeiro', 2, 2, 2, 2, 2, 2, 2, 2, 2, 
    800, 400, 80, 40
);

SELECT * FROM log_insercao;

# 2.6 Desenvolver, codificar e testar a aplicação, as consultas, as Views e demais objetos de banco

Passo 1: Desenvolver a aplicação em Python

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

# Conectar ao banco de dados
conn = sqlite3.connect('eleicoes.db')

# Função para executar uma consulta SQL e retornar o resultado como um DataFrame
def gerar_dataset(query):
    df = pd.read_sql_query(query, conn)
    return df

# Função para construir gráficos dinâmicos
def construir_grafico(df, x_col, y_col, grafico_tipo='bar'):
    if grafico_tipo == 'bar':
        df.plot(kind='bar', x=x_col, y=y_col)
    elif grafico_tipo == 'line':
        df.plot(kind='line', x=x_col, y=y_col)
    plt.show()

In [ ]:
def inserir_dados_perfil_eleitorado(dados):
    cursor = conn.cursor()
    cursor.executemany("""
        INSERT INTO PERFIL_ELEITORADO (
            DT_GERACAO, ANO_ELEICAO, SG_UF, NM_MUNICIPIO, NR_ZONA, CD_GENERO, 
            CD_ESTADO_CIVIL, CD_FAIXA_ETARIA, CD_GRAU_ESCOLARIDADE, CD_RACA_COR, 
            CD_IDENTIDADE_GENERO, CD_QUILOMBOLA, CD_INTERPRETE_LIBRAS, 
            QT_ELEITORES_PERFIL, QT_ELEITORES_BIOMETRIA, QT_ELEITORES_DEFICIENCIA, 
            QT_ELEITORES_INC_NM_SOCIAL
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, dados)
    conn.commit()

# Exemplo de uso
dados_exemplo = [
    ('2024-07-16', 2024, 'SP', 'Sao Paulo', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1000, 500, 100, 50),
    ('2024-07-16', 2024, 'RJ', 'Rio de Janeiro', 2, 2, 2, 2, 2, 2, 2, 2, 2, 800, 400, 80, 40)
]
inserir_dados_perfil_eleitorado(dados_exemplo)

In [ ]:
def gerar_relatorio_total_eleitores_por_estado():
    query = """
    SELECT SG_UF, SUM(QT_ELEITORES_PERFIL) AS total_eleitores
    FROM PERFIL_ELEITORADO
    GROUP BY SG_UF
    """
    df = gerar_dataset(query)
    print(df)
    construir_grafico(df, 'SG_UF', 'total_eleitores', 'bar')

# Exemplo de uso
gerar_relatorio_total_eleitores_por_estado()

Passo 2: Desenvolver e testar consultas SQL e Views

Criação de uma View para total de eleitores por estado

In [ ]:
CREATE VIEW IF NOT EXISTS total_eleitores_estado AS
SELECT 
    SG_UF, 
    SUM(QT_ELEITORES_PERFIL) AS total_eleitores
FROM 
    PERFIL_ELEITORADO
GROUP BY 
    SG_UF;


Testes SQL

In [ ]:
-- Testar a View total_eleitores_estado
SELECT * FROM total_eleitores_estado;

-- Testar inserção de dados na tabela PERFIL_ELEITORADO
INSERT INTO PERFIL_ELEITORADO (
    DT_GERACAO, ANO_ELEICAO, SG_UF, NM_MUNICIPIO, NR_ZONA, CD_GENERO, 
    CD_ESTADO_CIVIL, CD_FAIXA_ETARIA, CD_GRAU_ESCOLARIDADE, CD_RACA_COR, 
    CD_IDENTIDADE_GENERO, CD_QUILOMBOLA, CD_INTERPRETE_LIBRAS, 
    QT_ELEITORES_PERFIL, QT_ELEITORES_BIOMETRIA, QT_ELEITORES_DEFICIENCIA, 
    QT_ELEITORES_INC_NM_SOCIAL
) VALUES (
    '2024-07-16', 2024, 'MG', 'Belo Horizonte', 3, 3, 3, 3, 3, 3, 3, 3, 3, 
    600, 300, 60, 30
);

-- Verificar os dados inseridos
SELECT * FROM PERFIL_ELEITORADO WHERE SG_UF = 'MG';

-- Verificar se Trigger está funcionando corretamente
SELECT * FROM log_insercao;


Passo 3: Testar a aplicação e os objetos de banco de dados

Testes em Python

In [ ]:
def test_gerar_dataset():
    query = "SELECT * FROM PERFIL_ELEITORADO WHERE ANO_ELEICAO = 2024"
    dataset = gerar_dataset(query)
    assert not dataset.empty, "Dataset should not be empty"

def test_construir_grafico():
    df = pd.DataFrame({
        'SG_UF': ['SP', 'RJ', 'MG'],
        'total_eleitores': [1000000, 800000, 600000]
    })
    try:
        construir_grafico(df, 'SG_UF', 'total_eleitores', 'bar')
    except Exception as e:
        assert False, f"Grafico falhou com erro: {e}"

# Executar testes
test_gerar_dataset()
test_construir_grafico()


Testes SQL

In [ ]:
-- Testar a inserção e a visualização de dados na tabela PERFIL_ELEITORADO
INSERT INTO PERFIL_ELEITORADO (
    DT_GERACAO, ANO_ELEICAO, SG_UF, NM_MUNICIPIO, NR_ZONA, CD_GENERO, 
    CD_ESTADO_CIVIL, CD_FAIXA_ETARIA, CD_GRAU_ESCOLARIDADE, CD_RACA_COR, 
    CD_IDENTIDADE_GENERO, CD_QUILOMBOLA, CD_INTERPRETE_LIBRAS, 
    QT_ELEITORES_PERFIL, QT_ELEITORES_BIOMETRIA, QT_ELEITORES_DEFICIENCIA, 
    QT_ELEITORES_INC_NM_SOCIAL
) VALUES (
    '2024-07-16', 2024, 'BA', 'Salvador', 4, 4, 4, 4, 4, 4, 4, 4, 4, 
    700, 350, 70, 35
);

SELECT * FROM PERFIL_ELEITORADO WHERE SG_UF = 'BA';

-- Testar a View total_eleitores_estado
SELECT * FROM total_eleitores_estado WHERE SG_UF = 'BA';

-- Verificar o funcionamento da Trigger log_insercao_perfil
SELECT * FROM log_insercao WHERE SG_UF = 'BA';


# 2.7 Realizar a carga do banco de dados a partir dos dataset preparados

Feito pelo MySQL. 

#Douglas complementar, especificando como foi o processo passo a passo

# 2.8 Gerar os relatórios analíticos a serem entregues

In [ ]:
def gerar_relatorio_total_eleitores_estado():
    query = "SELECT * FROM total_eleitores_estado"
    df = gerar_dataset(query)
    print("Relatório: Total de Eleitores por Estado")
    print(df)
    construir_grafico(df, 'SG_UF', 'total_eleitores', 'bar')

gerar_relatorio_total_eleitores_estado()

In [ ]:
def gerar_relatorio_total_eleitores_municipio():
    query = "SELECT * FROM total_eleitores_municipio"
    df = gerar_dataset(query)
    print("Relatório: Total de Eleitores por Município")
    print(df)
    construir_grafico(df, 'NM_MUNICIPIO', 'total_eleitores', 'bar')

gerar_relatorio_total_eleitores_municipio()

In [ ]:
def gerar_relatorio_total_eleitores_faixa_etaria():
    query = "SELECT * FROM total_eleitores_faixa_etaria"
    df = gerar_dataset(query)
    print("Relatório: Total de Eleitores por Faixa Etária")
    print(df)
    construir_grafico(df, 'CD_FAIXA_ETARIA', 'total_eleitores', 'bar')

gerar_relatorio_total_eleitores_faixa_etaria()

In [ ]:
def gerar_relatorio_total_eleitores_escolaridade():
    query = "SELECT * FROM total_eleitores_escolaridade"
    df = gerar_dataset(query)
    print("Relatório: Total de Eleitores por Grau de Escolaridade")
    print(df)
    construir_grafico(df, 'CD_GRAU_ESCOLARIDADE', 'total_eleitores', 'bar')

gerar_relatorio_total_eleitores_escolaridade()

In [ ]:
def gerar_relatorio_total_eleitores_raca_cor():
    query = "SELECT * FROM total_eleitores_raca_cor"
    df = gerar_dataset(query)
    print("Relatório: Total de Eleitores por Raça/Cor")
    print(df)
    construir_grafico(df, 'CD_RACA_COR', 'total_eleitores', 'bar')

gerar_relatorio_total_eleitores_raca_cor()

In [ ]:
def gerar_relatorio_total_eleitores_deficiencia():
    query = "SELECT * FROM total_eleitores_deficiencia"
    df = gerar_dataset(query)
    print("Relatório: Total de Eleitores com Deficiência")
    print(df)

gerar_relatorio_total_eleitores_deficiencia()

Executar os relatórios para gerar as análises e os gráficos.

In [ ]:
# Executar todos os relatórios
gerar_relatorio_total_eleitores_estado()
gerar_relatorio_total_eleitores_municipio()
gerar_relatorio_total_eleitores_faixa_etaria()
gerar_relatorio_total_eleitores_escolaridade()
gerar_relatorio_total_eleitores_raca_cor()
gerar_relatorio_total_eleitores_deficiencia()